# Modelltraining für Pflanzenkrankheitserkennung

Dieses Notebook enthält den Code für das Training des Klassifikationsmodells zur Erkennung von Pflanzenkrankheiten.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml

# Konfiguration laden
with open('../config/model_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Pfade definieren
DATA_PATH = Path('../data/raw')
MODEL_PATH = Path('../models/classification_model')

## Datenvorverarbeitung und Datenaugmentation

In [ ]:
# Datenaugmentation definieren
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Datenladung
train_generator = train_datagen.flow_from_directory(
    DATA_PATH / 'train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    DATA_PATH / 'val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

## Modellarchitektur definieren

In [ ]:
# Transfer Learning mit vortrainiertem Modell
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Basis-Modell einfrieren
base_model.trainable = False

# Klassifikationsschichten hinzufügen
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 Klassen: healthy, disease_A, disease_B
])

# Modell kompilieren
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()